In [1]:
%load_ext autoreload
%autoreload 2

# SummedPauliOp as new PrimitiveOp

## Summary

`SummedPauliOp` is a wrapper class of `SparasePauliOp` in Terra's quantum_info.
This calss is functionally equivalent with `SummedOp` of `PauliOp` now, but the operations are fast.

This notebook is based on Qiskit Tutorials (now in [PR](https://github.com/Qiskit/qiskit-tutorials/pull/1068))

Link to 
- [this notebook](https://nbviewer.jupyter.org/github/ikkoham/qiskit-aqua/blob/sparsepauliop/qiskit/aqua/operators/primitive_ops/SummedPauliOp.ipynb).
- [Branch](https://github.com/ikkoham/qiskit-aqua/tree/sparsepauliop)
- [Diff](https://github.com/Qiskit/qiskit-aqua/compare/master...ikkoham:sparsepauliop)

## Progress and TODO
We have
- Implemented SummedPauliOp
- Supported PauliExpectation and Trotter.

However, we did not optimize the flow for `SparsePauliOp`.
We will improve
- TensoredOp
- AbelianGrouper, CircuitSampler, AerPauliExpectation
- EvolvedOp

We can support for `SummedPauliOp`
- Z2 Symmetry

TODO:

- Add Unittest

Commutator

In [2]:
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit.aqua.operators import PauliOp, SummedOp, SummedPauliOp, Zero
import numpy as np

In [3]:
xyz = SparsePauliOp(Pauli.from_label('XYZ'), coeffs=[2])
print(xyz)

SparsePauliOp([[False,  True,  True,  True,  True, False]],
              coeffs=[2.+0.j])


In [4]:
xxz = SparsePauliOp(Pauli.from_label('XXZ'), coeffs=[3])
print(xxz)

SparsePauliOp([[False,  True,  True,  True, False, False]],
              coeffs=[3.+0.j])


## Methods of SummedPauliOp

New `SummedPauliOp` supports methods which `PrimitiveOp` must have.

## SummedPauliOp, sums, compositions, and tensor products


In [5]:
# sum
print(SummedPauliOp(xyz) + SummedPauliOp(xxz))

SummedPauliOp([
2.0 * XYZ,
3.0 * XXZ,
])


In [6]:
# matrix product
print(SummedPauliOp(xyz) @ SummedPauliOp(xxz))

SummedPauliOp([
-6j * IZI,
])


In [7]:
# tensor product
print(SummedPauliOp(xyz) ^ SummedPauliOp(xxz))

SummedPauliOp([
6.0 * XYZXXZ,
])


In [8]:
from qiskit.aqua.operators import I, X, Y, Z

In [9]:
print(X + 2*Y)

SummedPauliOp([
1.0 * X,
2.0 * Y,
])


It used to be `SummedOp` of `PauliOp`, but now it will be changed to `SummedPauliOp`.

In [10]:
X + 2*Y

SummedPauliOp(SparsePauliOp([[ True, False],
               [ True,  True]],
              coeffs=[1.+0.j, 2.+0.j]), coeff=1)

In [11]:
print(((I-Z)^(I-Z)) + ((X-Y)^(X+Y)))

SummedPauliOp([
1.0 * II,
-1.0 * IZ,
-1.0 * ZI,
1.0 * ZZ,
1.0 * XX,
1.0 * XY,
-1.0 * YX,
-1.0 * YY,
])


### Constructor

In [12]:
SummedPauliOp(SparsePauliOp(Pauli(label='XYZ'), coeffs=[2]))

SummedPauliOp(SparsePauliOp([[False,  True,  True,  True,  True, False]],
              coeffs=[2.+0.j]), coeff=1.0)

### Eval

In [13]:
SummedPauliOp(xyz+xxz).eval('000')

DictStateFn({'011': (2+3j)}, coeff=1.0, is_measurement=False)

In [14]:
(SummedPauliOp(xyz+xxz) @ (Zero^3)).eval()

DictStateFn({'011': (2+3j)}, coeff=1.0, is_measurement=False)

In [15]:
SummedPauliOp(xyz+xxz).eval(Zero^3)

DictStateFn({'011': (2+3j)}, coeff=1.0, is_measurement=False)

### exp_i()

In [16]:
SummedOp(2*PauliOp(Pauli.from_label('XYZ'))).exp_i()

EvolvedOp(SummedOp(PauliOp(Pauli(z=[True, True, False], x=[False, True, True]), coeff=2.0), coeff=1.0, abelian=False), coeff=1.0)

In [17]:
SummedPauliOp(xyz).exp_i()

EvolvedOp(SummedPauliOp(SparsePauliOp([[False,  True,  True,  True,  True, False]],
              coeffs=[2.+0.j]), coeff=1.0), coeff=1.0)

### to_pauli_op()

In [18]:
SummedPauliOp(xyz+xxz).to_pauli_op()

SummedOp([PauliOp(Pauli(z=[True, True, False], x=[False, True, True]), coeff=2.0), PauliOp(Pauli(z=[True, False, False], x=[False, True, True]), coeff=3.0)], coeff=1.0, abelian=False)

### to_matrix()

In [19]:
SummedPauliOp(xyz).to_matrix()

array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.-2.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+2.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+2.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.-2.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.-2.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+2.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+2.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.-2.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

## Benchmark

In [20]:
from numpy.random import default_rng

rg = default_rng()

In [21]:
paulis = [Pauli.from_label(''.join(rg.choice(['I','X','Y','Z'], size=50))) for _ in range(300)]
weights = [rg.uniform() for _ in range(300)]

In [22]:
random_sp = SparsePauliOp.from_list(zip([pauli.to_label() for pauli in paulis], weights))

In [23]:
random_summed = SummedPauliOp(random_sp)

In [24]:
%%timeit

random_summed @ random_summed

44.2 ms ± 365 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Comparison to other classes,

WeightedPauliOperator:
CPU times: user 6.94 s, sys: 56.2 ms, total: 7 s
Wall time: 7.04 s

SummedOp([PauliOp]):
CPU times: user 2.96 s, sys: 30.1 ms, total: 2.99 s
Wall time: 3 s

## Expectations

In [25]:
two_qubit_H2 =  (-1.0523732 * I^I) + \
                (0.39793742 * I^Z) + \
                (-0.3979374 * Z^I) + \
                (-0.0112801 * Z^Z) + \
                (0.18093119 * X^X)

In [26]:
print(two_qubit_H2)

SummedPauliOp([
-1.0523732 * II,
0.39793742 * IZ,
-0.3979374 * ZI,
-0.0112801 * ZZ,
0.18093119 * XX,
])


In [27]:
two_qubit_H2

SummedPauliOp(SparsePauliOp([[False, False, False, False],
               [False, False,  True, False],
               [False, False, False,  True],
               [False, False,  True,  True],
               [ True,  True, False, False]],
              coeffs=[-1.0523732 +0.j,  0.39793742+0.j, -0.3979374 +0.j, -0.0112801 +0.j,
  0.18093119+0.j]), coeff=1)

In [28]:
from qiskit.aqua.operators import StateFn, PauliExpectation, CircuitSampler, CX, H, AerPauliExpectation
from qiskit import Aer

In [29]:
h2_measurement = StateFn(two_qubit_H2).adjoint()
print(h2_measurement)

OperatorMeasurement(SummedPauliOp([
-1.0523732 * II,
0.39793742 * IZ,
-0.3979374 * ZI,
-0.0112801 * ZZ,
0.18093119 * XX,
]))


In [30]:
print(PauliExpectation(group_paulis=False).convert(h2_measurement))

SummedOp([
  ComposedOp([
    OperatorMeasurement(-1.0523732 * II),
    II
  ]),
  ComposedOp([
    OperatorMeasurement(0.39793742 * IZ),
    II
  ]),
  ComposedOp([
    OperatorMeasurement(-0.3979374 * ZI),
    II
  ]),
  ComposedOp([
    OperatorMeasurement(-0.0112801 * ZZ),
    II
  ]),
  ComposedOp([
    OperatorMeasurement(0.18093119 * ZZ),
         ┌───┐
    q_0: ┤ H ├
         ├───┤
    q_1: ┤ H ├
         └───┘
  ])
])


In [31]:
print(PauliExpectation().convert(h2_measurement))

SummedOp([
  ComposedOp([
    OperatorMeasurement(AbelianSummedOp([
      -1.0523732 * II,
      0.18093119 * ZZ
    ])),
         ┌───┐
    q_0: ┤ H ├
         ├───┤
    q_1: ┤ H ├
         └───┘
  ]),
  ComposedOp([
    OperatorMeasurement(AbelianSummedOp([
      0.39793742 * IZ,
      -0.3979374 * ZI,
      -0.0112801 * ZZ
    ])),
    II
  ])
])


In [32]:
h2_pauli = PauliExpectation().convert(h2_measurement)
print(h2_pauli)

SummedOp([
  ComposedOp([
    OperatorMeasurement(AbelianSummedOp([
      -1.0523732 * II,
      0.18093119 * ZZ
    ])),
         ┌───┐
    q_0: ┤ H ├
         ├───┤
    q_1: ┤ H ├
         └───┘
  ]),
  ComposedOp([
    OperatorMeasurement(AbelianSummedOp([
      0.39793742 * IZ,
      -0.3979374 * ZI,
      -0.0112801 * ZZ
    ])),
    II
  ])
])


In [33]:
bell = CX @ (I ^ H) @ Zero
print(bell)

CircuitStateFn(
     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘
)


## Trotter

In [34]:
from qiskit.circuit import Parameter
from qiskit.aqua.operators import PauliTrotterEvolution, Suzuki

In [35]:
evo_time = Parameter('θ')
evolution_op = (evo_time*two_qubit_H2).exp_i()
print(evolution_op) # Note, EvolvedOps print as exponentiations
print(repr(evolution_op))

e^(-i*SummedPauliOp([
-1.0523732 * II,
0.39793742 * IZ,
-0.3979374 * ZI,
-0.0112801 * ZZ,
0.18093119 * XX,
]) * θ)
EvolvedOp(SummedPauliOp(SparsePauliOp([[False, False, False, False],
               [False, False,  True, False],
               [False, False, False,  True],
               [False, False,  True,  True],
               [ True,  True, False, False]],
              coeffs=[-1.0523732 +0.j,  0.39793742+0.j, -0.3979374 +0.j, -0.0112801 +0.j,
  0.18093119+0.j]), coeff=θ), coeff=1.0)


In [36]:
evo_and_meas = h2_measurement @ evolution_op @ bell
print(evo_and_meas)

ComposedOp([
  OperatorMeasurement(SummedPauliOp([
  -1.0523732 * II,
  0.39793742 * IZ,
  -0.3979374 * ZI,
  -0.0112801 * ZZ,
  0.18093119 * XX,
  ])),
  e^(-i*SummedPauliOp([
  -1.0523732 * II,
  0.39793742 * IZ,
  -0.3979374 * ZI,
  -0.0112801 * ZZ,
  0.18093119 * XX,
  ]) * θ),
  CircuitStateFn(
       ┌───┐     
  q_0: ┤ H ├──■──
       └───┘┌─┴─┐
  q_1: ─────┤ X ├
            └───┘
  )
])


In [37]:
trotterized_op = PauliTrotterEvolution(trotter_mode=Suzuki(order=2, reps=1)).convert(evo_and_meas)
# We can also set trotter_mode='suzuki' or leave it empty to default to first order Trotterization.
print(trotterized_op)

ComposedOp([
  OperatorMeasurement(SummedPauliOp([
  -1.0523732 * II,
  0.39793742 * IZ,
  -0.3979374 * ZI,
  -0.0112801 * ZZ,
  0.18093119 * XX,
  ])),
  CircuitStateFn(
  global phase: 1.0524
       ┌───┐     ┌───┐┌───┐┌──────────────────┐┌───┐┌───┐┌───┐»
  q_0: ┤ H ├──■──┤ H ├┤ X ├┤ RZ(0.18093119*θ) ├┤ X ├┤ H ├┤ X ├»
       └───┘┌─┴─┐├───┤└─┬─┘└──────────────────┘└─┬─┘├───┤└─┬─┘»
  q_1: ─────┤ X ├┤ H ├──■────────────────────────■──┤ H ├──■──»
            └───┘└───┘                              └───┘     »
  «     ┌──────────────────┐┌───┐┌──────────────────┐┌──────────────────┐┌───┐»
  «q_0: ┤ RZ(-0.0112801*θ) ├┤ X ├┤ RZ(0.39793742*θ) ├┤ RZ(0.39793742*θ) ├┤ X ├»
  «     └──────────────────┘└─┬─┘├──────────────────┤├──────────────────┤└─┬─┘»
  «q_1: ──────────────────────■──┤ RZ(-0.3979374*θ) ├┤ RZ(-0.3979374*θ) ├──■──»
  «                              └──────────────────┘└──────────────────┘     »
  «     ┌──────────────────┐┌───┐┌───┐┌───┐┌──────────────────┐┌───┐┌───┐
  «q_0: ┤ RZ

In [38]:
bound = trotterized_op.bind_parameters({evo_time: .5})

In [39]:
bound[1].to_circuit().draw()

global phase: 1.0524
     ┌───┐     ┌───┐┌───┐┌────────────────────────┐┌───┐┌───┐┌───┐»
q_0: ┤ H ├──■──┤ H ├┤ X ├┤ RZ(0.0904655950000000) ├┤ X ├┤ H ├┤ X ├»
     └───┘┌─┴─┐├───┤└─┬─┘└────────────────────────┘└─┬─┘├───┤└─┬─┘»
q_1: ─────┤ X ├┤ H ├──■──────────────────────────────■──┤ H ├──■──»
          └───┘└───┘                                    └───┘     »
«     ┌──────────────────────────┐┌───┐┌───────────────────────┐ »
«q_0: ┤ RZ(-0.00564005000000000) ├┤ X ├┤ RZ(0.198968710000000) ├─»
«     └──────────────────────────┘└─┬─┘├───────────────────────┴┐»
«q_1: ──────────────────────────────■──┤ RZ(-0.198968700000000) ├»
«                                      └────────────────────────┘»
«     ┌───────────────────────┐ ┌───┐┌──────────────────────────┐┌───┐┌───┐»
«q_0: ┤ RZ(0.198968710000000) ├─┤ X ├┤ RZ(-0.00564005000000000) ├┤ X ├┤ H ├»
«     ├───────────────────────┴┐└─┬─┘└──────────────────────────┘└─┬─┘├───┤»
«q_1: ┤ RZ(-0.198968700000000) ├──■────────────────────────────────■──┤ H ├»
«     └────────────────────────┘                                      └───┘»
«     ┌───┐┌────────────────────────┐┌───┐┌───┐
«q_0: ┤ X ├┤ RZ(0.0904655950000000) ├┤ X ├┤ H ├
«     └─┬─┘└────────────────────────┘└─┬─┘├───┤
«q_1: ──■──────────────────────────────■──┤ H ├
«                                         └───┘

In [40]:
diagonalized_meas_op = PauliExpectation().convert(trotterized_op)
print(diagonalized_meas_op)

SummedOp([
  ComposedOp([
    OperatorMeasurement(AbelianSummedOp([
      -1.0523732 * II,
      0.18093119 * ZZ
    ])),
    CircuitStateFn(
    global phase: 1.0524
         ┌───┐     ┌───┐┌───┐┌──────────────────┐┌───┐┌───┐┌───┐»
    q_0: ┤ H ├──■──┤ H ├┤ X ├┤ RZ(0.18093119*θ) ├┤ X ├┤ H ├┤ X ├»
         └───┘┌─┴─┐├───┤└─┬─┘└──────────────────┘└─┬─┘├───┤└─┬─┘»
    q_1: ─────┤ X ├┤ H ├──■────────────────────────■──┤ H ├──■──»
              └───┘└───┘                              └───┘     »
    «     ┌──────────────────┐┌───┐┌──────────────────┐┌──────────────────┐┌───┐»
    «q_0: ┤ RZ(-0.0112801*θ) ├┤ X ├┤ RZ(0.39793742*θ) ├┤ RZ(0.39793742*θ) ├┤ X ├»
    «     └──────────────────┘└─┬─┘├──────────────────┤├──────────────────┤└─┬─┘»
    «q_1: ──────────────────────■──┤ RZ(-0.3979374*θ) ├┤ RZ(-0.3979374*θ) ├──■──»
    «                              └──────────────────┘└──────────────────┘     »
    «     ┌──────────────────┐┌───┐┌───┐┌───┐┌──────────────────┐┌───┐┌───┐┌───┐
    «q_0: ┤ 

In [41]:
evo_time_points = list(range(8))
h2_trotter_expectations = diagonalized_meas_op.bind_parameters({evo_time: evo_time_points})

In [42]:
h2_trotter_expectations.eval()

array([-0.88272211-1.111e-15j, -0.88272211-1.165e-15j,
       -0.88272211-1.135e-15j, -0.88272211-1.178e-15j,
       -0.88272211-1.115e-15j, -0.88272211-8.360e-16j,
       -0.88272211-1.087e-15j, -0.88272211-1.161e-15j])

In [43]:
sampler = CircuitSampler(backend=Aer.get_backend('qasm_simulator'))
# sampler.quantum_instance.run_config.shots = 1000
sampled_trotter_exp_op = sampler.convert(h2_trotter_expectations)
sampled_trotter_energies = sampled_trotter_exp_op.eval()
print('Sampled Trotterized energies:\n {}'.format(np.real(sampled_trotter_energies)))

Sampled Trotterized energies:
 [-0.88272211 -0.88272211 -0.88272211 -0.88272211 -0.88272211 -0.88272211
 -0.88272211 -0.88272211]


In [44]:
print('Before:\n')
print(h2_trotter_expectations.reduce()[0][0])
print('\nAfter:\n')
print(sampled_trotter_exp_op[0][0])

Before:

ComposedOp([
  OperatorMeasurement(AbelianSummedOp([
    -1.0523732 * II,
    0.18093119 * ZZ
  ])),
  CircuitStateFn(
  global phase: 1.0524
       ┌───┐     ┌───┐┌───┐┌───────┐┌───┐┌───┐┌───┐┌───────┐┌───┐┌───────┐»
  q_0: ┤ H ├──■──┤ H ├┤ X ├┤ RZ(0) ├┤ X ├┤ H ├┤ X ├┤ RZ(0) ├┤ X ├┤ RZ(0) ├»
       └───┘┌─┴─┐├───┤└─┬─┘└───────┘└─┬─┘├───┤└─┬─┘└───────┘└─┬─┘├───────┤»
  q_1: ─────┤ X ├┤ H ├──■─────────────■──┤ H ├──■─────────────■──┤ RZ(0) ├»
            └───┘└───┘                   └───┘                   └───────┘»
  «     ┌───────┐┌───┐┌───────┐┌───┐┌───┐┌───┐┌───────┐┌───┐┌───┐┌───┐
  «q_0: ┤ RZ(0) ├┤ X ├┤ RZ(0) ├┤ X ├┤ H ├┤ X ├┤ RZ(0) ├┤ X ├┤ H ├┤ H ├
  «     ├───────┤└─┬─┘└───────┘└─┬─┘├───┤└─┬─┘└───────┘└─┬─┘├───┤├───┤
  «q_1: ┤ RZ(0) ├──■─────────────■──┤ H ├──■─────────────■──┤ H ├┤ H ├
  «     └───────┘                   └───┘                   └───┘└───┘
  )
])

After:

ComposedOp([
  OperatorMeasurement(AbelianSummedOp([
    -1.0523732 * II,
    0.18093119 * ZZ
  ])

## Benchmark of Sampler

In [45]:
sampler = CircuitSampler(backend=Aer.get_backend('qasm_simulator'))

sampled_exp_op = sampler.convert(h2_pauli @ bell)
sampled_energies = sampled_exp_op.eval()
print(f'Sampled energies:\n {sampled_energies.real}')

Sampled energies:
 -0.8827221099609374


In [46]:
#%%timeit 
#AerPauliExpectation().convert(h2_measurement)

sampler = CircuitSampler(backend=Aer.get_backend('statevector_simulator'))

sampled_exp_op = sampler.convert(h2_measurement @ bell)
sampled_energies2 = sampled_exp_op.eval()

In [47]:
print(f'Sampled energies:\n {sampled_energies2.real}')

Sampled energies:
 -0.88272211
